In [1]:
# Import Packages and libraries required

import pandas as pd
import numpy as np
import tensorflow as tf
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import copy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import GridSearchCV
import shap 
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import mean_absolute_error
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from itertools import cycle
import matplotlib.cm as cm
from geopy.geocoders import Nominatim
from geopy.distance import distance as gd
import pyproj
import requests
from openrouteservice import client
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

In [2]:
# Importing the data
crash_data = pd.read_csv('C:/Users/mural/Desktop/Traffic dataset/Updated ones/Traffic_Crashes_-_Crashes.csv',low_memory=False)
vehicle_data = pd.read_csv('C:/Users/mural/Desktop/Traffic dataset/Updated ones/Traffic_Crashes_-_Vehicles.csv',low_memory=False)
people_data = pd.read_csv('C:/Users/mural/Desktop/Traffic dataset/Updated ones/Traffic_Crashes_-_People.csv',low_memory=False)

In [3]:
# Merging the common columns
info1 = np.intersect1d(people_data.columns,(np.intersect1d(crash_data.columns, vehicle_data.columns)))
info2 = np.intersect1d(crash_data.columns, vehicle_data.columns)
info3 = np.intersect1d(people_data.columns, vehicle_data.columns)
print(info1,info2,info3)

['CRASH_DATE' 'CRASH_RECORD_ID' 'RD_NO'] ['CRASH_DATE' 'CRASH_RECORD_ID' 'RD_NO'] ['CRASH_DATE' 'CRASH_RECORD_ID' 'RD_NO' 'VEHICLE_ID']


In [4]:
# Converting the CRASH_DATE column to datetime data type

crash_data['CRASH_DATE'] = pd.to_datetime(crash_data['CRASH_DATE'])
vehicle_data['CRASH_DATE'] = pd.to_datetime(vehicle_data['CRASH_DATE'])
people_data['CRASH_DATE'] = pd.to_datetime(people_data['CRASH_DATE'])

In [5]:
# The 2021 data is only filtered 
Final_crash_data = crash_data[(crash_data['CRASH_DATE'] >= '2021-01-01') & (crash_data['CRASH_DATE'] <= '2021-12-31')]
Final_vehicle_data = vehicle_data[(vehicle_data['CRASH_DATE'] >= '2021-01-01') & (vehicle_data['CRASH_DATE'] <= '2021-12-31')]
Final_people_data = people_data[(people_data['CRASH_DATE'] >= '2021-01-01') & (people_data['CRASH_DATE'] <= '2021-12-31')]

In [6]:
# The values are merged using the common columns 
merged = pd.merge(left=Final_vehicle_data, right = Final_crash_data,left_on=['CRASH_RECORD_ID','RD_NO','CRASH_DATE'], right_on=["CRASH_RECORD_ID",'RD_NO','CRASH_DATE'])
merged_data = pd.merge(left=Final_people_data, right=merged,left_on = ['VEHICLE_ID','CRASH_RECORD_ID','RD_NO','CRASH_DATE'],right_on=['VEHICLE_ID','CRASH_RECORD_ID','RD_NO','CRASH_DATE'])


In [7]:
nulls = merged_data.isna().sum()
null_percent = nulls[nulls>0] / len(merged_data)
null_percent.to_frame('% Null').style.background_gradient(cmap='Blues')

,% Null
VEHICLE_ID,0.021553
SEAT_NO,0.801246
CITY,0.295012
STATE,0.282818
ZIPCODE,0.358507
SEX,0.016684
AGE,0.307975
DRIVERS_LICENSE_STATE,0.431188
DRIVERS_LICENSE_CLASS,0.544756
SAFETY_EQUIPMENT,0.003146


In [8]:
# extracting columns with missing values greater than 70%
Index_label = null_percent[null_percent>.70].index.tolist()

In [9]:
# Creating another variable which drops the columns in Index_label
merged_data_1 = merged_data.drop(columns = Index_label)

## AIM OF THE RESEARCH IS

### A) Predicting severity of a traffic accident
### B) Predicting factors which determine the severity of traffic accidents
### C) Provide Safer Route Based on the accident and Road Traffic Violation

In [10]:
# Creating a new variable 
severity_accident = copy.deepcopy(merged_data_1)

# A) Predicting severity of a traffic crashes

In [159]:
merged_data_1

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,CRASH_UNIT_ID,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,AREA_01_I,AREA_12_I,FIRST_CONTACT_POINT,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,O1241639,DRIVER,3af0e3ff1ea95965ea3be467aa58d0e74de4f9b11320be...,JE469682,1178465.0,2021-12-07 21:08:00,NaN,NaN,NaN,M,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,1241639,1,DRIVER,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),NaN,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN,UNKNOWN/NA,1.0,NaN,NaN,UNKNOWN,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,Y,"$501 - $1,500",12/07/2021 09:42:00 PM,NOT APPLICABLE,NOT APPLICABLE,1614,N,LARAMIE AVE,2532.0,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0,21,3,12,41.909977,-87.755823,POINT (-87.755822648009 41.909977009294)
1,O1123822,DRIVER,5f37bc99d63afdb5a967b8ac29c993bed6b1d36c705618...,JE267929,1065982.0,2021-06-16 09:00:00,MCCOOK,IL,NaN,M,50.0,NaN,NaN,SAFETY BELT USED,DEPLOYMENT UNKNOWN,NONE,"REPORTED, NOT EVIDENT",OTHER,OTHER,UNKNOWN,TEST NOT OFFERED,1123822,1,DRIVER,NaN,"ASSICUATED TRUCK & TRAILER (TIGARD, OR)","ASSICUATED TRUCK & TRAILER, TIGARD, OR",IL,2005.0,NONE,TRACTOR W/ SEMI-TRAILER,UNKNOWN/NA,S,STRAIGHT AHEAD,1.0,Y,NaN,FRONT,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,UNKNOWN INTERSECTION TYPE,STRAIGHT AND LEVEL,DRY,OTHER,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,NaN,"OVER $1,500",06/16/2021 09:30:00 AM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,5300,S,WESTERN AVE,923.0,1,"REPORTED, NOT EVIDENT",1.0,0.0,0.0,0.0,1.0,0.0,0.0,9,4,6,41.797395,-87.684273,POINT (-87.684272963451 41.797394747543)
2,O1250197,DRIVER,5d274b762c8fd736979d9c0cf8e28e59769a0816325059...,JE484633,1186661.0,2021-12-21 08:05:00,CHICAGO,IL,NaN,M,40.0,IL,D,USAGE UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NONE,UNKNOWN,NORMAL,TEST NOT OFFERED,1250197,2,DRIVER,1.0,LAND ROVER,LAND ROVER (SPORT),IL,2016.0,UNKNOWN,PASSENGER,PERSONAL,E,STRAIGHT AHEAD,2.0,NaN,NaN,ROOF,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,N,"OVER $1,500",12/21/2021 10:00:00 AM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,161,E,ERIE ST,1834.0,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,0.0,8,3,12,41.894049,-87.622999,POINT (-87.62299852501 41.894049259418)
3,P279567,PASSENGER,5d274b762c8fd736979d9c0cf8e28e59769a0816325059...,JE484633,1186661.0,2021-12-21 08:05:00,CHICAGO,IL,NaN,F,40.0,NaN,NaN,USAGE UNKNOWN,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,1250197,2,DRIVER,1.0,LAND ROVER,LAND ROVER (SPORT),IL,2016.0,UNKNOWN,PASSENGER,PERSONAL,E,STRAIGHT AHEAD,2.0,NaN,NaN,ROOF,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,N,"OVER $1,500",12/21/2021 10:00:00 AM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,161,E,ERIE ST,1834.0,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,0.0,8,3,12,41.894049,-87.622999,POINT (-87.62

In [11]:
# Removing interdependent columns
drop_list_1 = ['AREA_01_I','AREA_12_I']
severity_accident.drop(columns = drop_list_1, inplace = True)

In [12]:
# Removing columns with inconclusive details and lot of values similar to null

drop_list_2 = ['BAC_RESULT','EJECTION', 
            'BEAT_OF_OCCURRENCE','SEC_CONTRIBUTORY_CAUSE','REPORT_TYPE','VEHICLE_DEFECT']

severity_accident.drop(columns = drop_list_2, inplace = True)

#### Removing Null Values and Replacing them with neccessry values

In [13]:
#Removing Null and 0 values in the location 
severity_accident = severity_accident.dropna(subset=['LATITUDE','LONGITUDE','LOCATION'],axis=0)
severity_accident.dropna(subset=['LATITUDE', 'LONGITUDE','LOCATION'])
severity_accident.drop(severity_accident[severity_accident['LATITUDE']==0.0].index, inplace = True)

In [14]:
# Filling the Null values for HIT_AND_RUN_I

severity_accident.fillna({
    'HIT_AND_RUN_I':'N'
}, inplace=True)
severity_accident['HIT_AND_RUN_I'] = severity_accident.HIT_AND_RUN_I.replace({'Y':'1','N':'0'})

In [15]:
# Filling the Null values for SEX

severity_accident.fillna({
    'SEX':'X'
}, inplace=True)
severity_accident['SEX']

0         M
1         M
2         M
3         F
4         M
         ..
238063    M
238064    F
238065    M
238066    F
238067    F
Name: SEX, Length: 236258, dtype: object

In [16]:
# Filling the Null values for SAFETY_EQUIPMENT

severity_accident.fillna({
    'SAFETY_EQUIPMENT':'USAGE UNKNOWN'
}, inplace=True)
severity_accident['SAFETY_EQUIPMENT'].value_counts()

USAGE UNKNOWN                                 117819
SAFETY BELT USED                              104979
NONE PRESENT                                    7616
HELMET NOT USED                                 1521
SAFETY BELT NOT USED                            1136
CHILD RESTRAINT - FORWARD FACING                 909
BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)       654
CHILD RESTRAINT - TYPE UNKNOWN                   437
CHILD RESTRAINT - REAR FACING                    402
DOT COMPLIANT MOTORCYCLE HELMET                  276
BOOSTER SEAT                                     240
CHILD RESTRAINT NOT USED                          73
NOT DOT COMPLIANT MOTORCYCLE HELMET               60
SHOULD/LAP BELT USED IMPROPERLY                   54
WHEELCHAIR                                        44
CHILD RESTRAINT USED IMPROPERLY                   16
STRETCHER                                          9
HELMET USED                                        9
CHILD RESTRAINT USED                          

In [17]:
# Filling the Null values for AIRBAG_DEPLOYED

severity_accident.fillna({
    'AIRBAG_DEPLOYED':'DEPLOYMENT UNKNOWN'
}, inplace=True)
severity_accident['AIRBAG_DEPLOYED'].value_counts()

DID NOT DEPLOY                            99026
NOT APPLICABLE                            61492
DEPLOYMENT UNKNOWN                        55738
DEPLOYED, FRONT                            9193
DEPLOYED, COMBINATION                      7987
DEPLOYED, SIDE                             2667
DEPLOYED OTHER (KNEE, AIR, BELT, ETC.)      155
Name: AIRBAG_DEPLOYED, dtype: int64

In [18]:
# Filling the Null values for DRIVER_VISION

severity_accident.fillna({
    'DRIVER_VISION':'UNKNOWN'
}, inplace=True)
severity_accident['DRIVER_VISION'].value_counts()

UNKNOWN                   144567
NOT OBSCURED               87259
OTHER                       1882
MOVING VEHICLES             1000
PARKED VEHICLES              693
WINDSHIELD (WATER/ICE)       437
BLINDED - SUNLIGHT           214
TREES, PLANTS                 88
BUILDINGS                     53
BLINDED - HEADLIGHTS          20
EMBANKMENT                    18
HILLCREST                     15
BLOWING MATERIALS              9
SIGNBOARD                      3
Name: DRIVER_VISION, dtype: int64

In [19]:
# Filling the Null values for UNIT_TYPE

severity_accident.UNIT_TYPE.fillna(severity_accident.PERSON_TYPE,inplace=True)
severity_accident['UNIT_TYPE'].value_counts()

severity_accident.UNIT_TYPE

0         DRIVER
1         DRIVER
2         DRIVER
3         DRIVER
4         DRIVER
           ...  
238063    DRIVER
238064    DRIVER
238065    DRIVER
238066    DRIVER
238067    DRIVER
Name: UNIT_TYPE, Length: 236258, dtype: object

In [20]:
# Filling the Null values for VEHICLE_TYPE

severity_accident.fillna({
    'VEHICLE_TYPE':'UNKNOWN/NA'
}, inplace=True)

severity_accident['VEHICLE_TYPE'].value_counts()

PASSENGER                                 143746
SPORT UTILITY VEHICLE (SUV)                33938
UNKNOWN/NA                                 26549
VAN/MINI-VAN                               10161
PICKUP                                      7177
TRUCK - SINGLE UNIT                         4128
OTHER                                       3041
BUS OVER 15 PASS.                           2938
TRACTOR W/ SEMI-TRAILER                     1900
BUS UP TO 15 PASS.                           892
SINGLE UNIT TRUCK WITH TRAILER               569
MOTORCYCLE (OVER 150CC)                      547
OTHER VEHICLE WITH TRAILER                   248
TRACTOR W/O SEMI-TRAILER                     199
MOPED OR MOTORIZED BICYCLE                   123
ALL-TERRAIN VEHICLE (ATV)                     49
AUTOCYCLE                                     19
3-WHEELED MOTORCYCLE (2 REAR WHEELS)          17
FARM EQUIPMENT                                 8
RECREATIONAL OFF-HIGHWAY VEHICLE (ROV)         8
MOTOR DRIVEN CYCLE  

In [21]:
# Checking the Column Values to check if there have values that are Missing, Reduntant values or Values that cannot be converted

for columns in severity_accident.columns:
    print(columns,'\n',severity_accident[columns].value_counts())
    print('\n')
    print('Total Null Values: ',severity_accident[columns].isna().sum())
    print('\n')
print('\n') 

PERSON_ID 
 O1049256    5
O1049257    5
O1090857    4
O1067829    4
O1085042    4
           ..
P230709     1
O1133884    1
O1133792    1
O1133920    1
O1350810    1
Name: PERSON_ID, Length: 235578, dtype: int64


Total Null Values:  0


PERSON_TYPE 
 DRIVER                 184314
PASSENGER               46995
PEDESTRIAN               2984
BICYCLE                  1770
NON-MOTOR VEHICLE         159
NON-CONTACT VEHICLE        36
Name: PERSON_TYPE, dtype: int64


Total Null Values:  0


CRASH_RECORD_ID 
 ab683b4b18b6cc083b300ac34333e19e6f57b5a23b09d86524928d694ffe347a5aef5f9bf840aa4128ce389e4f234374560a08fe5ade4bdd69eb485d8c9165bd    28
ec14b8a3d84d0302ca0761d6e5ecc1c2dc335f8802f83a7c01b5141d8fffa5cb27d96b1e98207f2fa4f586acca7fdea268c41e7a51b3b4431a3d42c8f2238ed4    26
43bf767af796b06222eae2d7619dcd6ad0f144178da641010ad66c4b8b1ddc9f4e327c14d1e8a5db2c7bbe0662c31ceb61216f21684c69fea5473e8d90d6cb76    24
7e4048aa331ab03ec78f74e94e881ad33d776bc812a50005ce0aa4523ba2eb0087d89f0b70c46221dd39eba

In [22]:
# Removing columns that are Reduntant and values that cannot be converted as categorical

drop_list_3 = ['UNIT_NO',
 'VEHICLE_ID',
 'VEHICLE_USE',
 'VEHICLE_YEAR',
 'ZIPCODE','FIRST_CONTACT_POINT',
 'INJURY_CLASSIFICATION',
 'LIC_PLATE_STATE',
'DRIVERS_LICENSE_STATE',
'DRIVERS_LICENSE_CLASS',
 'MAKE',
 'MANEUVER',
 'MODEL',
 'NUM_PASSENGERS',
 'OCCUPANT_CNT',
 'PERSON_ID',
 'PERSON_TYPE',
 'PHYSICAL_CONDITION',
 'STATE','DATE_POLICE_NOTIFIED','TRAVEL_DIRECTION',
 'DRIVER_ACTION','CITY','AGE','CRASH_UNIT_ID']
severity_accident.drop(columns = drop_list_3, inplace = True)